# Bag of words model

In [117]:
import numpy as np

from torchtext import data
from torchtext import datasets
from torchtext.vocab import Vectors

import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.optim as optim
from torch.nn import Linear
from torch.nn.functional import softmax, relu, tanh
from torchtext.vocab import Vectors, GloVe, CharNGram, FastText

from sklearn.manifold import TSNE

from bokeh.plotting import figure, ColumnDataSource
from bokeh.models import HoverTool
from bokeh.io import output_notebook, show, push_notebook
output_notebook()


Loading BokehJS ...

In [2]:
use_cuda = torch.cuda.is_available()

def get_variable(x):
    """ Converts tensors to cuda, if available. """
    if use_cuda:
        return x.cuda()
    return x

def get_numpy(x):
    """ Get numpy array for both cuda and not. """
    if use_cuda:
        return x.cpu().data.numpy()
    return x.data.numpy()

## SST

In [239]:
TEXT = data.Field(sequential=True)
LABEL = data.Field(sequential=False)

train_set, validation_set, test_set = datasets.SST.splits(TEXT,LABEL, fine_grained=False, train_subtrees=True,filter_pred=lambda ex: ex.label != 'neutral')

In [240]:
print('train.fields', train_set.fields)
print('len(train)', len(train_set))
print('vars(train[0])', vars(train_set[0]))
print()
print('Example 2', vars(train_set[17]))

train.fields {'text': <torchtext.data.field.Field object at 0x0000020DCB2E4CC0>, 'label': <torchtext.data.field.Field object at 0x0000020DCB2E4898>}
len(train) 98794
vars(train[0]) {'text': ['The', 'Rock', 'is', 'destined', 'to', 'be', 'the', '21st', 'Century', "'s", 'new', '``', 'Conan', "''", 'and', 'that', 'he', "'s", 'going', 'to', 'make', 'a', 'splash', 'even', 'greater', 'than', 'Arnold', 'Schwarzenegger', ',', 'Jean-Claud', 'Van', 'Damme', 'or', 'Steven', 'Segal', '.'], 'label': 'positive'}

Example 2 {'text': ['The', 'gorgeously', 'elaborate', 'continuation', 'of', '``', 'The', 'Lord', 'of', 'the', 'Rings', "''", 'trilogy', 'is', 'so', 'huge', 'that', 'a', 'column', 'of', 'words', 'can', 'not', 'adequately', 'describe', 'co-writer\\/director', 'Peter', 'Jackson', "'s", 'expanded', 'vision', 'of', 'J.R.R.', 'Tolkien', "'s", 'Middle-earth', '.'], 'label': 'positive'}


In [241]:
# build the vocabulary
url = 'https://s3-us-west-1.amazonaws.com/fasttext-vectors/wiki.simple.vec'
TEXT.build_vocab(train_set, max_size=None, vectors=Vectors('wiki.simple.vec', url=url))
LABEL.build_vocab(train_set)
# print vocab information
print('len(TEXT.vocab)', len(TEXT.vocab))
print('TEXT.vocab.vectors.size()', TEXT.vocab.vectors.size())

# make iterator for splits
train_iter, val_iter, test_iter = data.BucketIterator.splits(
    (train_set, validation_set, test_set), batch_size=3)


# print batch information
batchsst = next(iter(train_iter))
print(batchsst.text)
print(batchsst.label)

len(TEXT.vocab) 18005
TEXT.vocab.vectors.size() torch.Size([18005, 300])
tensor([[2558,   21,   54],
        [  55, 9012,    1],
        [   3,  141,    1],
        [ 455,    7,    1],
        [5129,   13,    1],
        [   6,   31,    1],
        [ 350,   14,    1],
        [4728,   77,    1],
        [   1,    3,    1],
        [   1,  127,    1],
        [   1,   10,    1],
        [   1,  399,    1],
        [   1,    7,    1],
        [   1, 4765,    1],
        [   1,    3,    1],
        [   1,  553,    1],
        [   1,   55,    1],
        [   1,  782,    1],
        [   1,   13,    1],
        [   1,   46,    1],
        [   1,   14,    1],
        [   1, 2034,    1],
        [   1, 4677,    1],
        [   1,  334,    1],
        [   1,  174,    1],
        [   1,    3,    1],
        [   1,   20,    1],
        [   1,   10,    1],
        [   1,  503,    1],
        [   1,    7,    1],
        [   1,  137,    1],
        [   1,   33,    1],
        [   1,    5,    1],
   

In [242]:
embedding_dim = TEXT.vocab.vectors.size()[1]
num_embeddings = TEXT.vocab.vectors.size()[0]
num_classes = len(LABEL.vocab.itos)

# build the BoW model
class BoWNet(nn.Module):

    def __init__(self):
        super(BoWNet, self).__init__()
        self.embeddings = nn.Embedding(num_embeddings, embedding_dim)
        # use pretrained embeddings
        self.embeddings.weight.data.copy_(TEXT.vocab.vectors)
        
        # add hidden layers
        # YOUR CODE HERE!
        self.l_1 = Linear(in_features=600,
                           out_features=200,
                           bias = False)
        self.l_2 = Linear(in_features=200,
                           out_features=200,
                           bias=False)
        self.l_3 = Linear(in_features=200,
                           out_features = num_classes,
                           bias = False)
        
        # output layer
        self.l_out = Linear(in_features=num_classes,
                            out_features=num_classes,
                            bias=False)
        
    def forward(self, x, y):
        out = {}
        # get embeddings
        x = self.embeddings(x) # (bs,len,300)
        y = self.embeddings(y)
        
        print("X " + str(x.size()))
        print("Y " + str(y.size()))
        
        #sum_x =  # (bs,300) 
        
        print(x.size()[0])
        
        for i in range(0,x.size()[0]):
            sum_x += x
        
        #tanh # (bs,100)

    
        z = torch.cat((x,y),2)
        
        z = torch.tanh(self.l_1(z))
        
        z = torch.tanh(self.l_2(z))
        
        z = torch.tanh(self.l_3(z))
        
        #z = tanh(z)
        
        #z = Tanh(self.l_1(z))
        #z = Tanh(self.l_2(z))
        
        
        # mean embeddings, this is the bag of words trick
        out['bow'] = z = torch.mean(z, dim=0)
        
        # add hidden layers
        # YOUR CODE HERE!
#         out['l1_activations'] = x = relu(self.l_1(x))
#         out['l2_activations'] = x = relu(self.l_2(x))


        # Softmax
        out['out'] = softmax(self.l_out(z), dim=1)
        return out

net = BoWNet()
if use_cuda:
    net.cuda()
print(net)

BoWNet(
  (embeddings): Embedding(18005, 300)
  (l_1): Linear(in_features=600, out_features=200, bias=False)
  (l_2): Linear(in_features=200, out_features=200, bias=False)
  (l_3): Linear(in_features=200, out_features=3, bias=False)
  (l_out): Linear(in_features=3, out_features=3, bias=False)
)


In [223]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

def accuracy(ys, ts):
    # making a one-hot encoded vector of correct (1) and incorrect (0) predictions
    correct_prediction = torch.eq(torch.max(ys, 1)[1], ts)
    # averaging the one-hot encoded vector
    return torch.mean(correct_prediction.float())

In [224]:
def construct_sentences(batch):
    """    
    Parameters
    ----------
    batch: torchtext.data.batch.Batch
    
    Returns
    -------
    [str]
    """
    return [" ".join([TEXT.vocab.itos[elm] 
                      for elm in get_numpy(batch.text[:,i])])
            for i in range(batch.text.size()[1])]

def get_labels(batch):
    """
    Parameters
    ----------
    batch: torchtext.data.batch.Batch
    
    Returns
    -------
    [str]
    """
    return [LABEL.vocab.itos[get_numpy(batch.label[i])] for i in range(len(batch.label))]


In [225]:
derp = next(iter(val_iter))
print(derp.text[0])

C:\Users\Frederik\AppData\Roaming\Python\Python36\site-packages\torchtext\data\field.py:322: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  return Variable(arr, volatile=not train)


tensor([2034, 1189, 5204], device='cuda:0')


## TRAINING BOW WITH SST

In [226]:
max_iter = 1
eval_every = 1000
log_every = 200

# will be updated while iterating
#tsne_plot = show(p, notebook_handle=True)

train_loss, train_accs = [], []

net.train()
for i, batch in enumerate(train_iter):
    if i % eval_every == 0:
        net.eval()
        val_losses, val_accs, val_lengths = 0, 0, 0
        val_meta = {'label_idx': [], 'sentences': [], 'labels': []}
        for val_batch in val_iter:
            output = net(val_batch.text,val_batch.text)
            # batches sizes might vary, which is why we cannot just mean the batch's loss
            # we multiply the loss and accuracies with the batch's size,
            # to later divide by the total size
            val_losses += criterion(output['out'], val_batch.label) * val_batch.batch_size
            val_accs += accuracy(output['out'], val_batch.label) * val_batch.batch_size
            val_lengths += val_batch.batch_size
            
            for key, _val in output.items():
                if key not in val_meta:
                    val_meta[key] = []
                val_meta[key].append(get_numpy(_val)) 
            val_meta['label_idx'].append(get_numpy(val_batch.label))
            val_meta['sentences'].append(construct_sentences(val_batch))
            val_meta['labels'].append(get_labels(val_batch))
        
        for key, _val in val_meta.items():
            val_meta[key] = np.concatenate(_val)
        
        # divide by the total accumulated batch sizes
        val_losses /= val_lengths
        val_accs /= val_lengths
        
        print("valid, it: {} loss: {:.2f} accs: {:.2f}\n".format(i, get_numpy(val_losses), get_numpy(val_accs)))
        #update_plot(val_meta, 'bow', tsne_plot)
        
        net.train()
    
    output = net(batch.text,batch.text)
    batch_loss = criterion(output['out'], batch.label)
    
    train_loss.append(get_numpy(batch_loss))
    train_accs.append(get_numpy(accuracy(output['out'], batch.label)))
    
    optimizer.zero_grad()
    batch_loss.backward()
    optimizer.step()
    
    if i % log_every == 0:        
        print("train, it: {} loss: {:.2f} accs: {:.2f}".format(i, 
                                                               np.mean(train_loss), 
                                                               np.mean(train_accs)))
        # reset
        train_loss, train_accs = [], []
        
    if max_iter < i:
        break

C:\Users\Frederik\AppData\Roaming\Python\Python36\site-packages\torchtext\data\field.py:322: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  return Variable(arr, volatile=not train)


X torch.Size([3, 3, 300])
Y torch.Size([3, 3, 300])
X torch.Size([4, 3, 300])
Y torch.Size([4, 3, 300])
X torch.Size([5, 3, 300])
Y torch.Size([5, 3, 300])
X torch.Size([5, 3, 300])
Y torch.Size([5, 3, 300])
X torch.Size([5, 3, 300])
Y torch.Size([5, 3, 300])
X torch.Size([5, 3, 300])
Y torch.Size([5, 3, 300])
X torch.Size([5, 3, 300])
Y torch.Size([5, 3, 300])
X torch.Size([5, 3, 300])
Y torch.Size([5, 3, 300])
X torch.Size([6, 3, 300])
Y torch.Size([6, 3, 300])
X torch.Size([6, 3, 300])
Y torch.Size([6, 3, 300])
X torch.Size([6, 3, 300])
Y torch.Size([6, 3, 300])
X torch.Size([6, 3, 300])
Y torch.Size([6, 3, 300])
X torch.Size([6, 3, 300])
Y torch.Size([6, 3, 300])
X torch.Size([6, 3, 300])
Y torch.Size([6, 3, 300])
X torch.Size([6, 3, 300])
Y torch.Size([6, 3, 300])
X torch.Size([7, 3, 300])
Y torch.Size([7, 3, 300])
X torch.Size([7, 3, 300])
Y torch.Size([7, 3, 300])
X torch.Size([7, 3, 300])
Y torch.Size([7, 3, 300])
X torch.Size([7, 3, 300])
Y torch.Size([7, 3, 300])
X torch.Size

X torch.Size([20, 3, 300])
Y torch.Size([20, 3, 300])
X torch.Size([20, 3, 300])
Y torch.Size([20, 3, 300])
X torch.Size([20, 3, 300])
Y torch.Size([20, 3, 300])
X torch.Size([20, 3, 300])
Y torch.Size([20, 3, 300])
X torch.Size([20, 3, 300])
Y torch.Size([20, 3, 300])
X torch.Size([20, 3, 300])
Y torch.Size([20, 3, 300])
X torch.Size([20, 3, 300])
Y torch.Size([20, 3, 300])
X torch.Size([20, 3, 300])
Y torch.Size([20, 3, 300])
X torch.Size([20, 3, 300])
Y torch.Size([20, 3, 300])
X torch.Size([20, 3, 300])
Y torch.Size([20, 3, 300])
X torch.Size([20, 3, 300])
Y torch.Size([20, 3, 300])
X torch.Size([21, 3, 300])
Y torch.Size([21, 3, 300])
X torch.Size([21, 3, 300])
Y torch.Size([21, 3, 300])
X torch.Size([21, 3, 300])
Y torch.Size([21, 3, 300])
X torch.Size([21, 3, 300])
Y torch.Size([21, 3, 300])
X torch.Size([21, 3, 300])
Y torch.Size([21, 3, 300])
X torch.Size([21, 3, 300])
Y torch.Size([21, 3, 300])
X torch.Size([21, 3, 300])
Y torch.Size([21, 3, 300])
X torch.Size([21, 3, 300])
Y

## SNLI

In [243]:
print("Run test on SNLI...")
TEXT = datasets.snli.ParsedTextField()
LABEL = data.LabelField()
TREE = datasets.snli.ShiftReduceField()

train, val, test = datasets.SNLI.splits(TEXT, LABEL, TREE)

print("Fields:", train.fields)
print("Number of examples:\n", len(train))
print("First Example instance:\n", vars(train[0]))

TEXT.build_vocab(train)
LABEL.build_vocab(train)

train_iter, val_iter, test_iter = data.Iterator.splits((train, val, test), batch_size=3)

batch = next(iter(train_iter))
print("Numericalize premises:\n", batch.premise)
print("Numericalize hypotheses:\n", batch.hypothesis)
print("Entailment labels:\n", batch.label)

print("Test iters function")
train_iter, val_iter, test_iter = datasets.SNLI.iters(batch_size=3, trees=True)

batch = next(iter(train_iter))
print("Numericalize premises:\n", batch.premise)
print("Numericalize hypotheses:\n", batch.hypothesis)
print("Entailment labels:\n", batch.label)

val_iter_set = next(iter(val_iter))


Run test on SNLI...
Fields: {'premise': <torchtext.datasets.snli.ParsedTextField object at 0x0000020DA8EC2A20>, 'premise_transitions': <torchtext.datasets.snli.ShiftReduceField object at 0x0000020DA414FBA8>, 'hypothesis': <torchtext.datasets.snli.ParsedTextField object at 0x0000020DA8EC2A20>, 'hypothesis_transitions': <torchtext.datasets.snli.ShiftReduceField object at 0x0000020DA414FBA8>, 'label': <torchtext.data.field.LabelField object at 0x0000020E8148E320>}
Number of examples:
 549367
First Example instance:
 {'premise': ['A', 'person', 'on', 'a', 'horse', 'jumps', 'over', 'a', 'broken', 'down', 'airplane', '.'], 'premise_transitions': ['shift', 'shift', 'reduce', 'shift', 'shift', 'shift', 'reduce', 'reduce', 'reduce', 'shift', 'shift', 'shift', 'shift', 'shift', 'shift', 'reduce', 'reduce', 'reduce', 'reduce', 'reduce', 'shift', 'reduce', 'reduce'], 'hypothesis': ['A', 'person', 'is', 'training', 'his', 'horse', 'for', 'a', 'competition', '.'], 'hypothesis_transitions': ['shift',

C:\Users\Frederik\AppData\Roaming\Python\Python36\site-packages\torchtext\data\field.py:322: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  return Variable(arr, volatile=not train)


In [228]:
val_iter_set = next(iter(val_iter))


C:\Users\Frederik\AppData\Roaming\Python\Python36\site-packages\torchtext\data\field.py:322: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  return Variable(arr, volatile=not train)


In [229]:
print(val_iter_set.hypothesis.size()[0])
print(val_iter_set.premise.size()[0])



6
4


In [230]:
#sumP = 0
#sumH = 0

#for val_batch in val_iter:
#    print(val_batch.premise)
#    print(val_batch.hypothesis)
#    print(" ")
#    sumP += val_batch.premise.size()[0]
#    sumH += val_batch.hypothesis.size()[0]

C:\Users\Frederik\AppData\Roaming\Python\Python36\site-packages\torchtext\data\field.py:322: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  return Variable(arr, volatile=not train)


tensor([[    1,     1,     1],
        [  116,     1,     1],
        [   90,   457,   382],
        [   20, 21670, 10910]], device='cuda:0')
tensor([[   1,    1,    1],
        [   2,    2,    1],
        [  23,  457,    2],
        [  11,    3, 1419],
        [  90,    6,   11],
        [  14,   54,   54]], device='cuda:0')
 
tensor([[    1,     1,     1],
        [    1,   116,   116],
        [  457,    90,    90],
        [21670,    20,    20]], device='cuda:0')
tensor([[   1,    1,    1],
        [   2,    1,    1],
        [ 457,    1,    1],
        [3657,    2,    2],
        [   7,   23,  263],
        [  57,   11,   11],
        [  15, 1087,   90],
        [  14,   20,   14]], device='cuda:0')
 
tensor([[   1,    1,    1],
        [   2,    1,    1],
        [ 749, 2106, 2106],
        [ 391, 5278, 5278],
        [   4, 1106, 1106]], device='cuda:0')
tensor([[   1,    1,    1],
        [   1,    2,    1],
        [   1,  186,    2],
        [   1,    7,   65],
        [   1,

        [2250, 2250,    4]], device='cuda:0')
tensor([[   1,    1,    1],
        [   2,    2,    2],
        [  64, 3766,  119],
        [  51,    3,    7],
        [1547,   21,    9],
        [  18,    6,    6],
        [   6,  372,   34],
        [  54,   14,    4]], device='cuda:0')
 
tensor([[   1,    1,    1],
        [ 271,  271,    2],
        [  13,   13, 1012],
        [ 273,  273, 2703],
        [   8,    8, 4681],
        [1047, 1047,    4]], device='cuda:0')
tensor([[   1,    1,    1],
        [ 493,  396,    0],
        [  22,    3,    7],
        [  17,   18,  276],
        [5837,  176,  678],
        [   6,    6, 6403],
        [   8,    8,    8],
        [  14,   14,    4]], device='cuda:0')
 
tensor([[    1,     1,     1],
        [    2,     2,     1],
        [14183,  7652,   294],
        [   13,     5,     3],
        [  116,   508,     5],
        [   88,     0,    19],
        [   20,     4,   175]], device='cuda:0')
tensor([[  1,   1,   1],
        [  1,   1,  

        [    4,     4,     4]], device='cuda:0')
tensor([[   1,    1,    1],
        [   2,   40, 1114],
        [  43,    3,    6],
        [  11,   50,   48],
        [ 336, 1195,    7]], device='cuda:0')
 
tensor([[   1,    1,    1],
        [   2,    2,    2],
        [ 241,  310,  310],
        [   3,    3,    3],
        [  18,    9,    9],
        [5854, 4090, 4090],
        [  19,  174,  174],
        [ 175,    4,    4]], device='cuda:0')
tensor([[   1,    1,    1],
        [3581,  695, 5353],
        [  11,    3,    6],
        [  19,   91,  174],
        [   7,   54,   14]], device='cuda:0')
 
tensor([[   1,    1,    1],
        [   2,    2,    2],
        [ 850,  126, 1700],
        [   3,    7,    3],
        [  80,   39,   82],
        [  42,  671,  206],
        [  49,   88,  598],
        [  20,   20,    3]], device='cuda:0')
tensor([[  1,   1,   1],
        [  2,   2,   2],
        [ 42, 165, 206],
        [ 49,  11, 598],
        [ 20,  58,   4]], device='cuda:0')
 
te

        [  61,   61,   58]], device='cuda:0')
 
tensor([[   1,    1,    1],
        [   2,    2,    2],
        [3309,  115, 2334],
        [  31,    7,    3],
        [2323,    5,    9],
        [  24,   90, 7301],
        [  26,   32,   34],
        [   4,   20,    4]], device='cuda:0')
tensor([[   1,    1,    1],
        [   2,    2, 2334],
        [  43,   43,    3],
        [   6,   11, 2280],
        [  24,   90,   34],
        [  14,   20,    4]], device='cuda:0')
 
tensor([[   1,    1,    1],
        [   1,    1,    2],
        [ 454,    2,   52],
        [   7, 7652, 1061],
        [   9,    5,   68],
        [  15,  508,   70],
        [ 147,    0,   34],
        [  59,    4, 1996]], device='cuda:0')
tensor([[   1,    1,    1],
        [   2,    2,    1],
        [ 454, 5035,    2],
        [3556,   31,   70],
        [  37,  488,    6],
        [ 208,  508,   34],
        [ 298,    4,    4]], device='cuda:0')
 
tensor([[   1,    1,    1],
        [   2,    2,    2],
        

...

        [    4,     4,     4]], device='cuda:0')
tensor([[    1,     1,     1],
        [    1,     1,     2],
        [    2,   312,   161],
        [  214,     7,   244],
        [  483,     9,     3],
        [    3,   167,     5],
        [    5,   129,    33],
        [   33, 11740,     6],
        [   11,  2369,     8],
        [   58,     4,    14]], device='cuda:0')
 
tensor([[    1,     1,     1],
        [    2,     1,     1],
        [  102,     2,     2],
        [    3, 17458,  8438],
        [    5,  5937,    10],
        [  670, 16008,  2685],
        [   22,  2179,   718],
        [   12,     7,    12],
        [  149,  1598,    45],
        [    9,   949,     5],
        [  167,    35,   262],
        [  207,    67,    37],
        [   60,     3,     5],
        [   27,    17,    96],
        [   28,    76,    24],
        [    3,   105,    84],
        [    5,     7,    29],
        [   15,     5,    24],
        [   26,    82,    84],
        [   84,   705,    17],
 

        [   4,    4,   20]], device='cuda:0')
tensor([[   1,    1,    1],
        [   2,    1,    1],
        [ 193,    1,    1],
        [   3,    2,    1],
        [   9,  196,    2],
        [  96,    3,  188],
        [ 260,    9,   92],
        [  21,   25,    3],
        [3766,    3,   39],
        [   3,   17, 1275],
        [  29,   76,   11],
        [  52,  300,  292],
        [  86,    3, 9398],
        [   5,   72,  502],
        [  93,    6,    5],
        [  34,    8,   30],
        [   4,    4,   61]], device='cuda:0')
 
tensor([[   1,    1,    1],
        [   2,    2,    1],
        [ 458,  659,    2],
        [  52,    3,  498],
        [ 483,   77,   52],
        [   3, 1608,    7],
        [   5,    3,    5],
        [1947, 6618,   93],
        [ 434,  410,   34],
        [3744,   53,   32],
        [   3,    9,   67],
        [  10,  332,    3],
        [ 242,   62,   29],
        [  28,    0,   52],
        [   3,    3,   12],
        [  13,    5,  369],
        [ 

        [   54,     4,     4]], device='cuda:0')
tensor([[   1,    1,    1],
        [   2,    1,    1],
        [1069,   27,    1],
        [   9,    3,   27],
        [  11,    5,    3],
        [  30,   46,   69],
        [  61,    4,  504]], device='cuda:0')
 
tensor([[    1,     1,     1],
        [    2,     2,     2],
        [ 1470,    27,  1131],
        [ 2309,    28,  1054],
        [  194,     3,     7],
        [    3,     5,     9],
        [   17,     8,  1014],
        [ 3097,     3,    28],
        [   11,    50,   349],
        [  681,   390,  1098],
        [    7,    12,   660],
        [    5,   350,   461],
        [   51,     3,  1657],
        [    7,  2582,   534],
        [   29,   209,   249],
        [   16,    17,    22],
        [  280,   170,    77],
        [   12,     6,     0],
        [  525,    65,   114],
        [   67,     7,    69],
        [    3,    16,    10],
        [   50,    92,  1774],
        [24955,    22,     6],
        [    6,    12,

        [   61,     4,    61]], device='cuda:0')
tensor([[   1,    1,    1],
        [  49,    2,    1],
        [ 185,  529,  179],
        [  80,   22,    7],
        [   6,   13,    9],
        [ 429,  445,   36],
        [   7,  127,   49],
        [4846,    6,   51],
        [   8,    8,   11],
        [  14,   14,   54]], device='cuda:0')
 
tensor([[    1,     1,     1],
        [    2,     1,     1],
        [  320,     1,     1],
        [    7,     2,     2],
        [    9,    65,    65],
        [    6,     7,     7],
        [  390,     5,     5],
        [  611,  1039,  1039],
        [  106,    16,    16],
        [    5,   158,   158],
        [  198,    57,    57],
        [    3,     5,     5],
        [   29,   422,   422],
        [   16,    16,    16],
        [  256, 17356, 17356],
        [  361,    56,    56],
        [   71,    16,    16],
        [    3,    37,    37],
        [ 2094,    17,    17],
        [   17,    76,    76],
        [  296,    96,    96],


        [    4,    58,    14]], device='cuda:0')
tensor([[   1,    1,    1],
        [   1,    2,    1],
        [   1, 1470,    1],
        [   2,    3,    1],
        [1256,   17,    1],
        [ 337,  117,    1],
        [  69,  148,    1],
        [ 156,   11,    1],
        [ 416,  326,    1],
        [8767,   10,    1],
        [  10,   42,    1],
        [ 131,  326,    1],
        [   7,   16,    1],
        [ 276,   78,    1],
        [1439,    3,    1],
        [ 156,   80,    1],
        [  60,   86,    1],
        [ 295,   16,    1],
        [  22,   19,    2],
        [ 203,   12,  637],
        [  63,  565, 1973],
        [3016,    3,   11],
        [   8,   11,  833],
        [   4,   54,   14]], device='cuda:0')
 
tensor([[    1,     1,     1],
        [    2,     1,     1],
        [  335,     1,     1],
        [  355,     1,     1],
        [   69,     1,     1],
        [  261,     1,     1],
        [ 1357,     1,     1],
        [   60,     1,     1],
        [  

In [231]:
#print(sumP, sumH)

In [247]:
#print(TEXT.build_vocab.vectors.size())

AttributeError: 'function' object has no attribute 'vectors'

In [244]:
embedding_dim = TEXT.vocab.vectors.size()[1]
num_embeddings = TEXT.vocab.vectors.size()[0]
num_classes = len(LABEL.vocab.itos)

AttributeError: 'NoneType' object has no attribute 'size'

In [248]:
max_iter = 600
eval_every = 1000
log_every = 200

# will be updated while iterating
#tsne_plot = show(p, notebook_handle=True)

train_loss, train_accs = [], []

net.train()
for i, batch in enumerate(train_iter):
    if i % eval_every == 0:
        net.eval()
        val_losses, val_accs, val_lengths = 0, 0, 0
        val_meta = {'label_idx': [], 'sentences': [], 'labels': []}
        for val_batch in val_iter:
            output = net(val_batch.premise,val_iter_set.hypothesis)
            # batches sizes might vary, which is why we cannot just mean the batch's loss
            # we multiply the loss and accuracies with the batch's size,
            # to later divide by the total size
            val_losses += criterion(output['out'], val_batch.premise) * val_batch.batch_size
            val_accs += accuracy(output['out'], val_batch.label) * val_batch.batch_size
            val_lengths += val_batch.batch_size
            
            for key, _val in output.items():
                if key not in val_meta:
                    val_meta[key] = []
                val_meta[key].append(get_numpy(_val)) 
            val_meta['label_idx'].append(get_numpy(val_batch.label))
            val_meta['sentences'].append(construct_sentences(val_batch))
            val_meta['labels'].append(get_labels(val_batch))
        
        for key, _val in val_meta.items():
            val_meta[key] = np.concatenate(_val)
        
        # divide by the total accumulated batch sizes
        val_losses /= val_lengths
        val_accs /= val_lengths
        
        print("valid, it: {} loss: {:.2f} accs: {:.2f}\n".format(i, get_numpy(val_losses), get_numpy(val_accs)))
        #update_plot(val_meta, 'bow', tsne_plot)
        
        net.train()
    
    output = net(batch.premise,batch.hypothesis)
    batch_loss = criterion(output['out'], batch.label)
    
    train_loss.append(get_numpy(batch_loss))
    train_accs.append(get_numpy(accuracy(output['out'], batch.label)))
    
    optimizer.zero_grad()
    batch_loss.backward()
    optimizer.step()
    
    if i % log_every == 0:        
        print("train, it: {} loss: {:.2f} accs: {:.2f}".format(i, 
                                                               np.mean(train_loss), 
                                                               np.mean(train_accs)))
        # reset
        train_loss, train_accs = [], []
        
    if max_iter < i:
        break

C:\Users\Frederik\AppData\Roaming\Python\Python36\site-packages\torchtext\data\field.py:322: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  return Variable(arr, volatile=not train)


X torch.Size([4, 3, 300])
Y torch.Size([6, 3, 300])


TypeError: 'builtin_function_or_method' object is not subscriptable